In [ ]:
import random
from pprint import pprint

import scipy as sp
import spacy
import shap
import torch
import numpy as np
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
import matplotlib.pyplot as plt

from lib.utils import load_jsonl_file
from lib.lexicons import neutral_nouns, neutral_adverbs, neutral_verbs, bias_words

# nlp = spacy.load("en_core_web_trf")

# BATCH_SIZE = 16
CLASS_NAMES = ['monologic', 'dialogic']

# Load dataset
DATASET = load_jsonl_file("shared_data/dataset_1_6_1b_test.jsonl")
speech_class = [data for data in DATASET if data["label"] == "monologic"]
interview_class = [data for data in DATASET if data["label"] == "dialogic"]

# Load mismatched datapoint
mismatched_datapoints = load_jsonl_file("shared_data/dataset_1_8_2b_misclassified_examples.jsonl")
mismatched_datapoints = mismatched_datapoints[:1]


def get_device():
  """Returns the appropriate device available in the system: CUDA, MPS, or CPU"""
  if torch.backends.mps.is_available():
    return torch.device("mps")
  elif torch.cuda.is_available():
    return torch.device("cuda")
  else:
    return torch.device("cpu")


# Set device
device = get_device()
print(f"\nUsing device: {str(device).upper()}\n")

# Initialize constants
BERT_MODEL = 'bert-base-uncased'
# MODEL_PATH = 'models/1/paper_a_x_dl_bert_train_hop_bert.pth'
MODEL_PATH = 'models/1/paper_a_bert_solo.pth'

# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load the model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                                      num_labels=len(CLASS_NAMES),
                                                      hidden_dropout_prob=0.1)

# Move the model to the device
model = model.to(device)
# Load the model weights
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
# Set the model to evaluation mode
model.eval()


def f(x):
  tv = torch.tensor([tokenizer.encode(v, padding="max_length", max_length=500, truncation=True) for v in x], device=device)
  outputs = model(tv)[0].detach().cpu().numpy()
  scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
  val = sp.special.logit(scores[:, 1])  # use one vs rest logit units
  return val


# Initialize the SHAP explainer
# explainer = shap.Explainer(f, tokenizer, output_names=["monologic", "dialogic"])
explainer = shap.Explainer(f, tokenizer)

"""selected_texts = []
for mismatch in mismatched_datapoints:
  datapoint = None
  for data in DATASET:
    if data["id"] == mismatch["id"]:
      selected_texts.append(data["text"])
      break"""

texts = [data["text"] for data in speech_class]
# texts = [data["text"] for data in interview_class]


"""modified_tokens = []

doc = nlp(text)

# Replace bias words with neutral words
for token in doc:
  token_replaced = False
  # print(token.text, token.pos_, token.tag_)
  if token.lemma_ in bias_words and not token.is_stop and not token.is_punct:
    if token.pos_ == "NOUN":
      random_neutral_noun = random.choice(neutral_nouns)
      modified_tokens.append(random_neutral_noun + token.whitespace_)
      token_replaced = True
      # print(random_neutral_noun)
    elif token.pos_ == "VERB":
      random_neutral_verb = random.choice(neutral_verbs)
      modified_tokens.append(random_neutral_verb + token.whitespace_)
      token_replaced = True
    elif token.pos_ == "ADV":
      random_neutral_adverb = random.choice(neutral_adverbs)
      modified_tokens.append(random_neutral_adverb + token.whitespace_)
      token_replaced = True
  if not token_replaced:
    modified_tokens.append(token.text_with_ws)

modified_text = "".join(modified_tokens)"""

# Compute SHAP values for the selected samples
shap_values = explainer(texts, fixed_context=1)

# Visualize the SHAP values
# shap.plots.text(shap_values)

shap.plots.bar(shap_values.abs.sum(0), max_display=100, show=False)
# shap.plots.bar(shap_values.abs.max(0))

summed_shap_values = shap_values.abs.mean(0).values
summed_shap_features = shap_values.abs.mean(0).feature_names

# Convert to DataFrame for easy CSV saving
df_shap_values = pd.DataFrame(summed_shap_values, columns=["Value"])

# Add a column for the feature names if available
df_shap_values["Feature"] = summed_shap_features
df_shap_values = df_shap_values.sort_values(by="Value", ascending=False)

# Save to CSV
df_shap_values.to_csv("shared_data/dataset_1_9_speech_shap_features_plot_bar.csv", index=False)

# Save the plot PNG
plt.savefig("images/paper_a_20_speech_shap_features_plot_bar.png")


Loading data from shared_data/dataset_1_6_1b_test.jsonl...
Loaded 468 items.
Loading data from shared_data/dataset_1_8_2b_misclassified_examples.jsonl...
Loaded 9 items.

Using device: MPS


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

  0%|          | 0/498 [00:00<?, ?it/s]

PartitionExplainer explainer:   0%|          | 1/234 [00:00<?, ?it/s]

  0%|          | 0/498 [00:00<?, ?it/s]

...

  0%|          | 0/498 [00:00<?, ?it/s]

PartitionExplainer explainer:  79%|███████▊  | 184/234 [1:11:40<17:23, 20.88s/it]

  0%|          | 0/498 [00:00<?, ?it/s]

PartitionExplainer explainer:  79%|███████▉  | 185/234 [1:12:00<16:56, 20.75s/it]

  0%|          | 0/498 [00:00<?, ?it/s]